<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Helium_Mobius_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sympy as sp

# ==============================
# Parâmetros fixos
# ==============================
zeta = 1.8
Z = 2.0

# Variáveis simbólicas
u, v, w = sp.symbols('u v w', positive=True, real=True)

# Jacobiano em coordenadas perimétricas
J = sp.Rational(1,8)*u*v*w*(u+v)*(u+w)*(v+w)

# Expoente comum
expo = sp.exp(-2*zeta*u - zeta*v - zeta*w)

# ==============================
# Base Hylleraas N=2
# Singlet: i >= j
# Triplet: i > j
# ==============================
def hylleraas_basis(N, sector):
    B = []
    for i in range(N+1):
        for j in range(i+1):  # garante i>=j
            for k in range(N+1 - i - j):
                if sector == "triplet" and i == j:
                    continue
                B.append((i,j,k))
    return B

B_s = hylleraas_basis(2, "singlet")
B_t = hylleraas_basis(2, "triplet")

# ==============================
# Função φ(i,j,k) = (r1^i)(r2^j)(r12^k) e^{-zeta(r1+r2)}
# Nas coords perimétricas:
# r1 = (u+v)/2, r2 = (u+w)/2, r12 = (v+w)/2
# ==============================
r1 = (u+v)/2
r2 = (u+w)/2
r12= (v+w)/2

def phi(i,j,k):
    return (r1**i)*(r2**j)*(r12**k)*sp.exp(-zeta*(r1+r2))

# ==============================
# Integrais de sobreposição S_ab = ∫ φ_a φ_b J d(u,v,w)
# Aqui deixamos tudo simbólico primeiro e avaliamos numericamente no fim.
# ==============================
def overlap_element(b1, b2):
    i1,j1,k1 = b1
    i2,j2,k2 = b2
    integrand = phi(i1,j1,k1)*phi(i2,j2,k2)*J
    val = sp.integrate(sp.integrate(sp.integrate(
        integrand,
        (u,0,sp.oo)), (v,0,sp.oo)), (w,0,sp.oo))
    return sp.simplify(val)

# Montagem das matrizes S_singlet e S_triplet
def build_S(basis):
    n = len(basis)
    S = sp.zeros(n)
    for a in range(n):
        for b in range(a, n):
            sab = overlap_element(basis[a], basis[b])
            S[a,b] = sab
            S[b,a] = sab
    return S

print("Base singlet:", B_s)
S_sing = build_S(B_s)
print("\nMatriz S_singlet (ζ=1.8):")
sp.pprint(S_sing)

print("\nBase triplet:", B_t)
S_tri = build_S(B_t)
print("\nMatriz S_triplet (ζ=1.8):")
sp.pprint(S_tri)

import sympy as sp

# ==============================
# Parâmetros fixos
# ==============================
zeta = 1.8
Z = 2.0

# Variáveis simbólicas
u, v, w = sp.symbols('u v w', positive=True, real=True)

# Coordenadas perimétricas → r1, r2, r12
r1  = (u+v)/2
r2  = (u+w)/2
r12 = (v+w)/2

# Jacobiano completo
J = sp.Rational(1,8) * u*v*w * (u+v)*(u+w)*(v+w)

# Fator exponencial que surge em φ_a φ_b
expo = sp.exp(-2*zeta*u - zeta*v - zeta*w)

# ==============================
# Base Hylleraas N=2
# ==============================
def hylleraas_basis(N, sector):
    B = []
    for i in range(N+1):
        for j in range(i+1):  # i>=j
            for k in range(N+1 - i - j):
                if sector == "triplet" and i == j:
                    continue
                B.append((i,j,k))
    return B

B_s = hylleraas_basis(2, "singlet")
B_t = hylleraas_basis(2, "triplet")

# ==============================
# φ(i,j,k) = r1^i r2^j r12^k e^{-ζ(r1+r2)}
# ==============================
def phi(i,j,k):
    return (r1**i)*(r2**j)*(r12**k)*sp.exp(-zeta*(r1+r2))

# Combinação simetrizada/antissimetrizada (i>=j sempre na base)
def Phi_sym(i,j,k, sector):
    if sector == "singlet":
        if i == j:
            return phi(i,j,k)
        # (φ_ijk + φ_jik)/√2
        return (phi(i,j,k) + phi(j,i,k))/sp.sqrt(2)
    else:
        # triplet: só i>j existem na base; (φ_ijk − φ_jik)/√2
        return (phi(i,j,k) - phi(j,i,k))/sp.sqrt(2)

# ==============================
# Construção da matriz S (para referência/consistência)
# ==============================
def overlap_matrix(basis, sector):
    n = len(basis)
    S = sp.zeros(n)
    for a,(i,j,k) in enumerate(basis):
        Pa = Phi_sym(i,j,k, sector)
        for b,(p,q,r) in enumerate(basis):
            if b < a:
                S[a,b] = S[b,a]
                continue
            Pb = Phi_sym(p,q,r, sector)
            integrand = Pa*Pb*J
            Sab = sp.integrate(sp.integrate(sp.integrate(
                integrand, (u,0,sp.oo)), (v,0,sp.oo)), (w,0,sp.oo))
            S[a,b] = sp.simplify(Sab)
            S[b,a] = S[a,b]
    return S

print("Base singlet:", B_s)
S_sing = overlap_matrix(B_s, "singlet")
print("\nMatriz S_singlet (ζ=1.8):")
sp.pprint(S_sing)

print("\nBase triplet:", B_t)
S_tri = overlap_matrix(B_t, "triplet")
print("\nMatriz S_triplet (ζ=1.8):")
sp.pprint(S_tri)

# ==============================
# Construção da matriz V (sem denominadores)
# V = -Z(1/r1 + 1/r2) + 1/r12
# 1/r1 = 2/(u+v), 1/r2 = 2/(u+w), 1/r12 = 2/(v+w)
# Com J = (1/8) u v w (u+v)(u+w)(v+w),
#   J/(u+v) = (1/8) u v w (u+w)(v+w)   (polinómio puro)
#   J/(u+w) = (1/8) u v w (u+v)(v+w)
#   J/(v+w) = (1/8) u v w (u+v)(u+w)
# ==============================
J_uv = sp.Rational(1,8) * u*v*w * (u+w)*(v+w)  # J/(u+v)
J_uw = sp.Rational(1,8) * u*v*w * (u+v)*(v+w)  # J/(u+w)
J_vw = sp.Rational(1,8) * u*v*w * (u+v)*(u+w)  # J/(v+w)

def potential_matrix(basis, sector):
    n = len(basis)
    V = sp.zeros(n)
    for a,(i,j,k) in enumerate(basis):
        Pa = Phi_sym(i,j,k, sector)
        for b,(p,q,r) in enumerate(basis):
            if b < a:
                V[a,b] = V[b,a]
                continue
            Pb = Phi_sym(p,q,r, sector)
            # V integrand (sem denominadores):
            # -2Z * Pa*Pb*J_uv  - 2Z * Pa*Pb*J_uw  + 2 * Pa*Pb*J_vw
            integrand = (-2*Z)*Pa*Pb*J_uv + (-2*Z)*Pa*Pb*J_uw + (2)*Pa*Pb*J_vw
            # NOTA: Pa*Pb já contém exp(-ζ(r1+r2))^2 → expo correto
            Vab = sp.integrate(sp.integrate(sp.integrate(
                integrand, (u,0,sp.oo)), (v,0,sp.oo)), (w,0,sp.oo))
            V[a,b] = sp.simplify(Vab)
            V[b,a] = V[a,b]
    return V

print("\nConstruindo V_singlet (ζ=1.8, Z=2.0)…")
V_sing = potential_matrix(B_s, "singlet")
print("Matriz V_singlet:")
sp.pprint(V_sing)

print("\nConstruindo V_triplet (ζ=1.8, Z=2.0)…")
V_tri = potential_matrix(B_t, "triplet")
print("Matriz V_triplet:")
sp.pprint(V_tri)


Base singlet: [(0, 0, 0), (0, 0, 1), (0, 0, 2), (1, 0, 0), (1, 0, 1), (1, 1, 0), (2, 0, 0)]

Matriz S_singlet (ζ=1.8):
⎡0.00787712033603881  0.0139162459270019  0.0287369019666601  0.00984640042004 ↪
⎢                                                                              ↪
⎢0.0139162459270019   0.0287369019666601  0.0677902382828854  0.01932811934305 ↪
⎢                                                                              ↪
⎢0.0287369019666601   0.0677902382828854  0.179639404096907   0.04390360022684 ↪
⎢                                                                              ↪
⎢0.00984640042004851  0.0193281193430582  0.0439036002268418  0.01487900507918 ↪
⎢                                                                              ↪
⎢0.0193281193430582   0.0439036002268418  0.112983730471476   0.03233509273744 ↪
⎢                                                                              ↪
⎢0.0124721071987281   0.026723049699679   0.0658115034933969  0.0190546